<a href="https://colab.research.google.com/github/dprBloo/YBIGTA-HW-/blob/master/%ED%81%AC%EB%A1%A4%EB%A7%81%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 크롤링 실습

In [1]:
# 필요한 라이브러리 다운로드
# 이미 설치되어 있으면 실행하지 않으셔도 됩니다

!pip3 install requests
!pip3 install bs4
!pip3 install selenium

     |████████████████████████████████| 904 kB 4.1 MB/s 


In [2]:
import requests
from bs4 import BeautifulSoup


url = 'http://www.cine21.com/rank/person/content'

In [3]:
def actors_info(page):
    post_data = {}
    post_data['section'] = 'actor'
    post_data['period_start'] = '2021-07'
    post_data['gender'] = 'all'
    post_data['page'] = page
    
    res = requests.post(url, data=post_data)
    soup = BeautifulSoup(res.text, 'html.parser')
    actors = soup.select('li.people_li div.name')
    scores = soup.select('li.people_li > ul.num_info > li')
    movies = soup.select('ul.mov_list span')
    ranks = soup.select('span.grade')
    idx = 0
    actors_info_list = []
    
    for i in range(len(actors)):
        actor_info_dict = {}
        
        actor = actors[i].text
        name, appeared = actor[:actor.find('(')], int(actor[actor.find('(') + 1:-2])
        actor_info_dict['이름'] = name
        
        score = scores[i]
        actor_info_dict['흥행지수'] = int(score.strong.text.replace(',', ''))
        
        appear = []
        for _ in range(appeared):
            appear.append(movies[idx].text)
            idx += 1
        actor_info_dict['출연영화'] = appear
        
        actor_info_dict['랭킹'] = ranks[i].text
        
        link = 'http://www.cine21.com' + actors[i].a['href']
        res_actor = requests.get(link)
        soup_actor = BeautifulSoup(res_actor.text, 'html.parser')
        details = soup_actor.select_one('ul.default_info').select('li')
        
        for detail in details:
            key = detail.span.text
            value = detail.text.replace('\n', '')[len(key):]
            actor_info_dict[key] = value
        
        actors_info_list.append(actor_info_dict)
    
    return actors_info_list

In [4]:
result = []

for page in range(1, 11):
    result.extend(actors_info(page))

result

[{'랭킹': '1',
  '생년월일': '1968-01-21',
  '성별': '남',
  '소속사': '심엔터테인먼트',
  '신장/체중': '178cm, 68kg',
  '이름': '김윤석',
  '직업': '배우',
  '출연영화': ['모가디슈'],
  '취미': '여행, 낚시',
  '학교': '동의대학교 학사',
  '흥행지수': 138669},
 {'다른 이름': '趙寅成',
  '랭킹': '2',
  '생년월일': '1981-07-28',
  '성별': '남',
  '소속사': '싸이더스 HQ',
  '신장/체중': '186cm, 72kg',
  '이름': '조인성',
  '직업': '배우',
  '출연영화': ['모가디슈'],
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '흥행지수': 121335},
 {'랭킹': '3',
  '생년월일': '1964-03-03',
  '성별': '남',
  '소속사': '지티비엔터테인먼트',
  '신장/체중': '180cm, 75kg',
  '원어명': '許俊豪',
  '이름': '허준호',
  '직업': '배우',
  '출연영화': ['모가디슈', '국가부도의 날'],
  '취미': '만화책 보기',
  '특기': '야구, 농구',
  '학교': '서울예술대학 연극과',
  '흥행지수': 104003},
 {'랭킹': '4',
  '성별': '여',
  '이름': '김소진',
  '직업': '배우',
  '출연영화': ['모가디슈', '아이 캔 스피크'],
  '흥행지수': 86677},
 {'랭킹': '5',
  '성별': '남',
  '이름': '정만식',
  '직업': '배우',
  '출연영화': ['모가디슈'],
  '흥행지수': 69334},
 {'다른 이름': '조신제',
  '랭킹': '6',
  '생년월일': '1979-01-16',
  '성별': '남',
  '이름'

### 크롤링 주소 요청
dictionary를 이용하여 form data 생성   
request모듈의 post 함수를 이용하여 요청 전송

- 첫 번째 인자: 요청 url
- 두 번째 인자: form data

In [ ]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2021-07'
post_data['gender'] = 'all'
post_data['page'] = 1

res = requests.post(cine21_url, data=post_data)

In [ ]:
res.text

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')

### 배우 이름 추출

In [ ]:
# select(): find_all()과 동일한 작동을 하는 함수
# 차이점은 파라미터로 css selector를 사용
# 좀 더 간편하다
soup.select('li.people_li div.name') # class가 people_li인 li 태그 중 class가 name인 div 태그 list

In [ ]:
actors = soup.select('li.people_li div.name')
for actor in actors:
    print(actor.text)

이름만 추출할 수 없을까? -> 정규표현식 사용

In [ ]:
test_data = '조우진(2편)'
# re.sub(정규 표현식, 치환 문자, 대상 문자열)
re.sub('\(\w*\)', '', test_data) # \w: 문자, 숫자와 매치

In [ ]:
actors = soup.select('li.people_li div.name')
for actor in actors:
    print(re.sub('\(\w*\)', '', actor.text))

### 배우 상세 정보 추출

In [ ]:
actors = soup.select('li.people_li div.name')
# select_one(): find()와 대응되는 함수
for actor in actors:
    print (actor.select_one('a'))

In [ ]:
actors = soup.select('li.people_li div.name')
actors_info_list = list()

for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.select_one('a')['href'] # 태그 내 속성 접근
    response_actor = requests.get(actor_link) # 배우 상세 페이지 이동
    soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    print(actor_details)

배우마다 가지고 있는 정보가 다르다!   
-> dictionary 형태로 저장

In [ ]:
actors = soup.select('li.people_li div.name')
actors_info_list = list()

for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.select_one('a')['href'] # 태그 내 속성 접근
    response_actor = requests.get(actor_link) # 배우 상세 페이지 이동
    soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    actor_info_dict = dict()
    
    for actor_detail in actor_details:
        actor_detail_key = actor_detail.select_one('span.tit').text # 상세정보 key값
        actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail)) # span 태그 제거
        actor_detail_value = re.sub('<.*?>', '', actor_detail_value) # 상세정보 value값
        actor_info_dict[actor_detail_key] = actor_detail_value
        
    actors_info_list.append(actor_info_dict)

actors_info_list

### 특수한 정규 표현식
Greedy(.*) vs Non-Greedy(.*?)
- . 문자는 개행 문자를 제외한 모든 문자 한개를 의미
- \* 문자는 앞 문자가 0번 이상 반복되는 패턴
- ? 문자는 패턴에 여러 개가 포함될 경우 가장 적은 개수만 포함되도록 함

```html
<li><span class="tit">직업</span>배우</li>
  - <.*> -> <li><span class="tit">원어명</span>주지훈</li>
  - <.*?> -> <li>
```

### 최종 코드

In [ ]:
actors_info_list = list()
actors = soup.select('li.people_li div.name')

for index, actor in enumerate(actors):
    actor_info_dict = dict()
    actor_name = re.sub('\(\w*\)', '', actor.text)
    actor_info_dict['이름'] = actor_name
    
    actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    for actor_detail in actor_details:
        actor_detail_key = actor_detail.select_one('span.tit').text
        actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail))
        actor_detail_value = re.sub('<.*?>', '', actor_detail_value)
        actor_info_dict[actor_detail_key] = actor_detail_value
        
    actors_info_list.append(actor_info_dict)

actors_info_list